# Import Libraries

In [5]:
!pip install pyswarms

import glob
import os
import pandas as pd
import numpy as np
import pyswarms as ps
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE # Dùng để cân bằng dữ liệu

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization, Conv1D, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Kết nối Google Drive
from google.colab import drive
drive.mount('/content/drive')

# CẤU HÌNH ĐƯỜNG DẪN
# Hãy thay đổi đường dẫn này trỏ đến thư mục chứa dataset của bạn trên Drive
DATASET_PATH = "/content/drive/My Drive/Colab Notebooks/Model_on_pi/data/raw/CSE-CIC-IDS2018"
SAMPLE_PER_FILE = 50000 # Lấy 50.000 mẫu mỗi file để xử lý

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 3.4 MB/s eta 0:00:00
Mounted at /content/drive


# Load And Clean Data

In [ ]:
def load_and_clean_data(folder_path, sample_size):
    all_files = glob.glob(os.path.join(folder_path, "*.csv"))

    processed_frames = []

    # Loại bỏ một vài code không cần thiết trước khi chạy LKPSO
    cols_to_drop = [
        'Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Destination Port', 'Timestamp',
        'Bwd PSH Flags', 'Bwd URG Flags', 'Fwd URG Flags', 'CWE Flag Count',
        'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate',
        'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate',
        'Subflow Fwd Pkts', 'Subflow Fwd Byts', 'Subflow Bwd Pkts', 'Subflow Bwd Byts',
        'Fwd Header Length.1'
    ]

    for file in all_files:
        try:
            # Đọc file với tùy chọn low_memory=False để tránh cảnh báo
            df = pd.read_csv(file, low_memory=False)
            df.columns = df.columns.str.strip() # Xóa khoảng trắng tên cột

            # Lấy mẫu ngẫu nhiên nếu file quá lớn
            if len(df) > sample_size:
                df_sampled = df.sample(n=sample_size, random_state=42)
            else:
                df_sampled = df

            # Xóa các cột không quan trọng khi nó tồn tại trong bộ dữ liệu
            existing_cols = [c for c in cols_to_drop if c in df_sampled.columns]
            df_sampled = df_sampled.drop(columns=existing_cols)

            # Xử lý NaN/Inf (Thay thế vô cực bằng NaN rồi xóa dòng NaN)
            df_sampled = df_sampled.replace([np.inf, -np.inf], np.nan).dropna()

            processed_frames.append(df_sampled)
            print(f"Đã xử lý: {os.path.basename(file)} | Shape: {df_sampled.shape}")

            del df # Giải phóng RAM

        except Exception as e:
            print(f"Lỗi đọc file {file}: {e}")

    # Gộp tất cả dữ liệu lại
    if processed_frames:
        print("Đang ghép nối dữ liệu...")
        combined_df = pd.concat(processed_frames, ignore_index=True)
        print(f"Tổng kích thước dữ liệu sau gộp: {combined_df.shape}")
        return combined_df
    else:
        return None

# Gọi hàm tải dữ liệu
df = load_and_clean_data(DATASET_PATH, SAMPLE_PER_FILE)

# Kiểm tra và tách X, y
if df is not None and 'Label' in df.columns:
    X = df.drop('Label', axis=1)
    y = df['Label']

    # Mã hóa nhãn (Label Encoding)
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    num_classes = len(le.classes_)
    print("Các lớp tấn công tìm thấy:", le.classes_)
else:
    print("Lỗi dữ liệu hoặc không tìm thấy cột Label")
    exit()

Đã xử lý: DoS attacks-Slowloris.csv | Shape: (36754, 71)
Đã xử lý: DDOS attack-LOIC-UDP.csv | Shape: (5784, 71)
Đã xử lý: Brute Force -XSS.csv | Shape: (734, 71)
Đã xử lý: SQL Injection.csv | Shape: (286, 71)
Đã xử lý: Brute Force -Web.csv | Shape: (2073, 71)
Đã xử lý: DoS attacks-GoldenEye.csv | Shape: (50000, 71)
Đã xử lý: DoS attacks-SlowHTTPTest.csv | Shape: (50000, 71)
Đã xử lý: Bot.csv | Shape: (50000, 71)
Đã xử lý: DDOS attack-HOIC.csv | Shape: (50000, 71)
Đã xử lý: Infilteration.csv | Shape: (50000, 71)


# Using Likely Point PSO to choose the most important features

In [ ]:
print("Đang chạy Feature Selection với LPPSO")

# Lấy một mẫu nhỏ để chạy PSO cho nhanh
X_pso, _, y_pso, _ = train_test_split(X, y_encoded, train_size=20000, stratify=y_encoded, random_state=42)

# Hàm Fitness cho PSO (Mục tiêu: Tăng Accuracy, Giảm số lượng Feature)
def f_per_particle(m, alpha=0.9):
    if np.count_nonzero(m) == 0:
        return 1.0 # Phạt nếu không chọn feature nào

    # Chỉ lấy các cột được chọn (m[i] == 1)
    X_subset = X_pso.iloc[:, m == 1]

    # Dùng RandomForest nhẹ để đánh giá độ hiệu quả của tập feature này
    clf = RandomForestClassifier(n_estimators=20, n_jobs=-1, random_state=42)
    clf.fit(X_subset, y_pso)
    acc = clf.score(X_subset, y_pso)

    # Hàm mục tiêu (Cost function) cần giảm thiểu
    n_features = X_subset.shape[1]
    total_features = X_pso.shape[1]
    j = (alpha * (1.0 - acc)) + ((1.0 - alpha) * (n_features / total_features))
    return j

def f(x):
    n_particles = x.shape[0]
    j = [f_per_particle(x[i] > 0.5) for i in range(n_particles)]
    return np.array(j)

# Cấu hình PSO
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
dimensions = X.shape[1]
optimizer = ps.discrete.BinaryPSO(n_particles=15, dimensions=dimensions, options=options)

# Chạy tối ưu hóa
cost, pos = optimizer.optimize(f, iters=15)

# Lấy danh sách đặc trưng đã chọn
selected_indices = np.where(pos == 1)[0]
SELECTED_FEATURES = X.columns[selected_indices].tolist()

print(f"Đã chọn được {len(SELECTED_FEATURES)} đặc trưng quan trọng nhất:")
print(SELECTED_FEATURES)

# Prepare data

In [ ]:
print("Chuẩn bị dữ liệu huấn luyện")

# Chỉ giữ lại các đặc trưng PSO đã chọn
X_final = X[SELECTED_FEATURES]

# Cân bằng dữ liệu bằng SMOTE (Optional nhưng tốt cho các lớp thiểu số)
try:
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_final, y_encoded)
except Exception as e:
    print(f"{e}")
    X_resampled, y_resampled = X_final, y_encoded

# Chia tập Train/Test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled
)

# Chuẩn hóa Min-Max
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Lưu scaler để dùng sau này trên Raspberry Pi
joblib.dump(scaler, 'scaler.pkl')

# Reshape cho mô hình (Samples, Timesteps=1, Features)
# CNN-LSTM cần đầu vào 3 chiều
X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])

# One-hot encoding cho label
y_train_onehot = to_categorical(y_train)
y_test_onehot = to_categorical(y_test)

print(f"Input Shape cho mô hình: {X_train_reshaped.shape}")

# Create hybrid model

In [ ]:
def create_hybrid_model(input_shape, num_classes):
    model = Sequential()

    # 1D_CNN (Trích xuất đặc trưng không gian)
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=1))

    # LSTM (Học phụ thuộc thời gian)
    model.add(LSTM(units=128, return_sequences=False))
    model.add(Dropout(0.3))

    # Output (Phân loại)
    model.add(Dense(units=num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Tạo mô hình
input_shape = (X_train_reshaped.shape[1], X_train_reshaped.shape[2])
model = create_hybrid_model(input_shape, num_classes)
model.summary()

# Thiết lập callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('model_hybrid_pso.keras', monitor='val_accuracy', save_best_only=True)

# Main

In [ ]:
history = model.fit(
    X_train_reshaped, y_train_onehot,
    epochs=15,
    batch_size=70,
    validation_data=(X_test_reshaped, y_test_onehot),
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

# Đánh giá
print("Đánh giá mô hình trên tập Test...")
evaluation = model.evaluate(X_test_reshaped, y_test_onehot, verbose=0)
print(f"Test Loss: {evaluation[0]:.4f}")
print(f"Test Accuracy: {evaluation[1] * 100:.2f}%")

# Vẽ biểu đồ
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label="Training Accuracy")
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label="Training Loss")
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title("Model Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()

In [ ]:
def Main():
    print("Traning completed!")

Main()